In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import theano.tensor as tt
import warnings

RANDOM_SEED = 20090425

%matplotlib inline
sns.set()
warnings.simplefilter(action='ignore', category=FutureWarning)
print(f'Running on PyMC3 v{pm.__version__} and Pandas v{pd.__version__}')

Running on PyMC3 v3.6 and Pandas v0.24.1


#### Import dependent variable (election results) and standardized regressors (there are 7 of them)

In [2]:
X_full = pd.read_json('X_full.json')
X_full = X_full.set_index(['departement', 'date']).sort_index()
X_full.head()

climat_affaires  conf_menages  prix_gazole  inflation  \
departement date                                                             
ain         dep1992        -0.408263      0.315657    -1.460156   2.114784   
            dep1994        -0.601886      0.116938    -1.249131   0.733849   
            dep1998         0.976141      0.514375    -1.002935  -0.396006   
            dep2001         1.237532      2.402197    -0.545714   0.231691   
            dep2004         0.492084     -0.081780    -0.510543   0.733849   

                          pib   net_app   chomage  
departement date                                   
ain         dep1992  0.323604 -0.286913 -1.852963  
            dep1994 -0.363523  0.093029 -0.676859  
            dep1998  1.455350  0.743212 -1.068894  
            dep2001  0.962999  1.172080 -2.392010  
            dep2004  0.856889  0.459211 -1.607941

In [3]:
dptmts_idx, dptmts_names = X_full.index.get_level_values(0).factorize(sort=True)
n_dptmts = len(dptmts_names)
date_idx, date_names = X_full.index.get_level_values(1).factorize(sort=True)
_, n_regressors = X_full.shape
X_full.shape

(1889, 7)

In [4]:
# For now, we ignore elections where one of these parties is not competing
results = pd.read_json('Y_full.json')
results = results.set_index(['departement', 'date']).sort_index()
results.head()

farleft    left   green  center   right  farright   other
departement date                                                              
ain         dep1992   0.0482  0.1389  0.0741  0.2488  0.2064    0.1259  0.1577
            dep1994   0.0698  0.1216  0.0548  0.2617  0.1476    0.1149  0.2296
            dep1998   0.0732  0.1161  0.0267  0.2524  0.1473    0.1508  0.2335
            dep2001   0.0717  0.1221  0.0531  0.0240  0.1211    0.0817  0.5263
            dep2004   0.0461  0.1084  0.0315  0.0171  0.2411    0.1484  0.4074

In [5]:
y_full = (results.copy() * 10000).astype(int) # transform proportions to multinomial observations
_, n_parties = y_full.shape
y_full.head()

farleft  left  green  center  right  farright  other
departement date                                                         
ain         dep1992      482  1389    741    2488   2064      1259   1577
            dep1994      698  1216    548    2617   1476      1149   2296
            dep1998      732  1161    267    2524   1473      1508   2335
            dep2001      717  1221    531     240   1211       817   5263
            dep2004      461  1084    315     171   2411      1484   4074

In [6]:
# make sure there is no missing data here
# national regressors: fixed by department but varying per election
groupX = X_full.loc['nord', :].iloc[:, :n_regressors-1].values

#### This model samples quite nicely

In [9]:
with pm.Model() as hier_model:
    
    sigma_dpt_interp = pm.HalfNormal('sigma_dpt_interp', 100.)
    sigma_fixed = pm.HalfNormal('sigma_fixed', 100.)
    sigma_random = pm.HalfNormal('sigma_random', 100.)
    
    dpt_interp = pm.Normal('dpt_interp', 0., sigma_dpt_interp, shape=(n_dptmts, n_parties-1))
    fixed_effect = pm.Normal('fixed_effect', 0., sigma_fixed, shape=(n_regressors-1, n_parties-1))
    random_effect = pm.Normal('random_effect', 0., sigma_random, shape=n_parties-1)
    
    results_est = dpt_interp[dptmts_idx]\
                    + tt.dot(groupX, fixed_effect)[date_idx]\
                    + X_full['chomage'].values[:, None] * random_effect
    results_est = tt.concatenate(tensor_list=[results_est, 
                                              tt.zeros((X_full.shape[0], 1))], 
                                 axis=1)
    
    probs = pm.Deterministic('probs', tt.nnet.softmax(results_est))
    likelihood = pm.Multinomial('likelihood',
                                n=y_full.sum(axis=1),
                                p=probs,
                                observed=y_full.values)
    # might need to increase tunning
    trace = pm.sample(1000, tune=2000, cores=4, random_seed=RANDOM_SEED)

In [ ]:
hier_model.check_test_point()

##### No divergences, good BFMI, good energy

In [ ]:
print("BFMI (values smaller than 0.2 indicate poor sampling): ", np.round(pm.bfmi(trace), 3))

In [ ]:
pm.energyplot(trace, figsize=(10,6));

In [ ]:
pm.traceplot(trace, varnames=['sigma_dpt_interp', 'sigma_fixed', 'sigma_random', 
                              'random_effect', 'fixed_effect', 'dpt_interp', 'oos_probs'], 
             figsize=(14, 14));

##### But the inference seems biased when looking at ppcs

In [ ]:
sim_data = pm.sample_posterior_predictive(trace=trace, samples=2000, model=hier_model, random_seed=RANDOM_SEED)
rdm_pt = np.random.randint(len(y_full) + 1)
reals, simuls = y_full.iloc[rdm_pt].values, sim_data['likelihood'][:, rdm_pt, :].T

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(12,16))
axes_flat = axes.flatten()

for ax, real, sim in zip(axes_flat, reals, simuls):
    ax.hist(sim, bins=20)
    ax.vlines(real, *ax.get_ylim(), colors='red', label='Real result')
    ax.set_yticklabels([])
    sns.despine(left=True)
    ax.legend();

plt.tight_layout();

**Where does the bias come from?**
- Too many regressors?
- The softmax applies too strong a normalization? But how do you get rid of the softmax/multinomial likelihood in this case?
- The data actually contain another cluster - the type of election ('dep', 'euro', 'pres', etc.). Would modeling it improve the model? But how can we implement that in PyMC?